<a href="https://colab.research.google.com/github/p5149247263/Tensorflow/blob/main/Data_Validation_with_TensorFlow_Data_Validation_(TFDV).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Stage 1: Install all dependencies and setting up the environment

In [ ]:
!apt-get install python-dev python-snappy

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-dev is already the newest version (2.7.15~rc1-1).
The following NEW packages will be installed:
  python-snappy
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 10.8 kB of archives.
After this operation, 39.9 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python-snappy amd64 0.5-1.1build2 [10.8 kB]
Fetched 10.8 kB in 0s (35.6 kB/s)
Selecting previously unselected package python-snappy.
(Reading database ... 160772 files and directories currently installed.)
Preparing to unpack .../python-snappy_0.5-1.1build2_amd64.deb ...
Unpacking python-snappy (0.5-1.1build2) ...
Setting up python-snappy (0.5-1.1build2) ...


In [ ]:
!pip install tensorflow_data_validation

## Stage 2: Import project dependencies

In [ ]:
import pandas as pd
import tensorflow as tf
import tensorflow_data_validation

from __future__ import print_function

AttributeError: ignored

In [ ]:
from apache_beam  import Metrics

ImportError: ignored

In [ ]:
!pip install apache-beam

In [ ]:
!pip install cachetools --upgrade

Requirement already up-to-date: cachetools in /usr/local/lib/python3.7/dist-packages (4.2.2)


In [ ]:
import tensorflow_data_validation

AttributeError: ignored

## Stage 3: Simple dataset analysis

In [ ]:
import pandas as pd
dataset = pd.read_csv("pollution_small.csv")

In [ ]:
dataset.shape

(2188, 5)

In [ ]:
training_data = dataset[:1600]

In [ ]:
training_data.describe()

,pm10,no2,so2,soot
count,1600.000000,1600.000000,1600.000000,1600.000000
mean,49.656494,30.980519,16.229981,21.551956
std,35.211906,12.400788,10.621896,12.127354
min,6.380000,9.740000,4.010000,6.000000
25%,28.345000,22.567500,9.777500,14.400000
50%,38.835000,28.715000,13.275000,18.630000
75%,58.050000,36.370000,19.282500,24.072500
max,277.250000,138.010000,123.130000,107.650000


In [ ]:
test_set = dataset[1600:]

In [ ]:
test_set.describe()

,pm10,no2,so2,soot
count,588.000000,588.000000,588.000000,588.000000
mean,44.648248,37.296922,13.605170,18.441310
std,28.992087,10.940050,5.098944,6.596459
min,11.900000,15.070000,4.990000,8.000000
25%,28.337500,29.217500,10.122500,14.410000
50%,35.555000,35.815000,12.345000,17.090000
75%,50.812500,43.872500,15.855000,20.962500
max,273.770000,106.030000,38.030000,87.210000


## Stage 3: Data analysis and validation with TFDV

### Generate training data statistics

In [ ]:
train_stats = tfdv.generate_statistics_from_dataframe(dataframe=dataset)

NameError: ignored

### Infering the schema

In [ ]:
schema = tfdv.infer_schema(statistics=train_stats)

In [ ]:
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Date',BYTES,required,,-
'so2',FLOAT,required,,-
'pm10',FLOAT,required,,-
'no2',FLOAT,required,,-
'soot',FLOAT,required,,-


### Calculate test set statistics

In [ ]:
test_stats = tfdv.generate_statistics_from_dataframe(dataframe=test_set)

## Stage 4: Compare test statistics with the Schema

### Checking for anomalies in new data

In [ ]:
anomalies = tfdv.validate_statistics(statistics=test_stats, schema=schema)

### Displaying all detected anomalies

- Integer larger than 10
- STRING type when expected INT type
- FLOAT type when expected INT type
- Integer smaller than 0

In [ ]:
tfdv.display_anomalies(anomalies)

### New data WITH anomalies

In [ ]:
test_set_copy = test_set.copy()

In [ ]:
test_set_copy.drop("soot", axis=1, inplace=True)

### Statistics based on data with anomalies

In [ ]:
test_set_copy_stats = tfdv.generate_statistics_from_dataframe(dataframe=test_set_copy)

In [ ]:
anomalies_new = tfdv.validate_statistics(statistics=test_set_copy_stats, schema=schema)

In [ ]:
tfdv.display_anomalies(anomalies_new)

,Anomaly short description,Anomaly long description
Feature name,,
'soot',Column dropped,Column is completely missing


## Stage 5: Prepare the schema for Serving

In [ ]:
schema.default_environment.append("TRAINING")
schema.default_environment.append("SERVING")

### Removing a target column from the Serving schema

In [ ]:
tfdv.get_feature(schema, "soot").not_in_environment.append("SERVING")

### Checking for anomalies between the SERVING environment and new test set

In [ ]:
serving_env_anomalies = tfdv.validate_statistics(test_set_copy_stats, schema, environment="SERVING")

In [ ]:
tfdv.display_anomalies(serving_env_anomalies)